In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Basic K-NN with Titanic DataSet

https://www.kaggle.com/competitions/titanic/data

In [ ]:
#get data
!git clone https://github.com/s3nnahoj/dsLabData.git

In [ ]:
DATA_FILE_TRAIN = 'dsLabData/titanic_train.csv' # we only use the annotated data yet
df = pd.read_csv(DATA_FILE_TRAIN, header=0, index_col='PassengerId')
df.loc[df.Age.isnull(),'Age'] = df.Age.median() # fill nan's in age

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.hist('Sex',by='Survived')

In [ ]:
df.groupby('Survived').Fare.hist(legend=True, alpha =0.5)

### Train / Test Split

In [ ]:
dfTrain=df.sample(frac=0.8,random_state=42)
dfTest=df.drop(dfTrain.index)

In [ ]:
dfTest

### KNN-Classification

In [ ]:
def extractFeatureVector(dfRow):
    gender = 0
    if dfRow.Sex == "female":
        gender = 1
    age = (dfRow.Age - dfTrain.Age.mean()) / dfTrain.Age.std()
    return np.array([gender,age])

In [ ]:
df.loc[1]

In [ ]:
extractFeatureVector(df.loc[1])

In [ ]:
trainData = []
trainLabel = []

for idx, row in dfTrain.iterrows():
    trainData.append(extractFeatureVector(row))
    trainLabel.append(row.Survived)

In [ ]:
def predict(x,k=3):
    distances = [] #(distance,class)
    for idx,tX in enumerate(trainData):
        d = np.linalg.norm(tX-x,ord=1)
        distances.append((d,trainLabel[idx]))
    distances.sort()
    countSurvived = 0
    for i in range(0,k):
        countSurvived+=distances[i][1]
        #print("nn %d with distance %.2f: y=%d" % (i,distances[i][0],distances[i][1]))
    
    if countSurvived >= (k/2.0):
        return 1
    else:
        return 0


In [ ]:
predict(extractFeatureVector(dfTest.iloc[0]))    

## Evaluation

In [ ]:
countCorrect = 0
for idx, row in dfTest.iterrows():
   result = predict(extractFeatureVector(row))
   if result == row.Survived:
      countCorrect+=1

print("correct predictions: %d / %d , accuracy: %.2f " %(countCorrect ,len(dfTest),countCorrect / len(dfTest)))

# Assignment 4 - Titanic and KNN



1 ) Exploratory data analysis: Further explore the Titanic data set by summarizing statiscis and data visualization. In particular focus on:
  
  * distribution of the individual features. are they quantitative or qualitative? what about the possible range of values?
  * class conditional distributions. how are the features distributed for the survived / not survived case. Can you related to the Baysian decision theory?
  * which features are most discriminative?

2) Extend the given KNN implementation by adding more features to the feature vector. Play around and evaluate your changes. Can you increase the accuracy on the given split? 